# Moving Forward
* Options for structuring kernel
* More hardware
* Non-determinism
* Model to handle non-determinism

## Function Interactions in Kernel
Lots of moving components
* eg. Process/Thread admin, virtual memory, device management, IPC, scheduler, etc.

# Structuring Techniques

## Monolithic
* Everything in one big program
* Efficient
* Hard to maintain
* Large mememory requirement
* Hard to customize

## Microkernel
* Functionality split into a bunch of system processes
* Smaller memory requirement
* Easier to maintain and customize
* Less efficient due to context switching

# Kernel Responsibilities
* Manipulate process state and hardware
* Guarantee system integrity
* Communicate with processes through well-defined interfaces (ie. syscalls and events)
* Manage resources

## Organization by layering
1. HW
2. Memory Manager
3. Process Manager
4. Process coordination (semaphores)
5. IPC
6. Real time clock
7. Device managers, drivers
8. Inter-machine communication
9. Filesystem
10. User programs

## System Components
* Bootstrap program
* Kernel
* Persistent data structures (eg. filesystem)
* Daemons (system processes)
* User processes

## CPU Parts and Functionality
* Registers
* Instructions for load/manip/store
* Instruction ptr
* Stack + ptr
* Flag registers
* Interrupt control unit
* Memory management unit
* Device access mechanism

# Create Processes and Process Switching

## x86 Architecture

Various notes from the rambling in class

**Processor**
* "User Registers" - registers everyone has access to
    * ESP - stack ptr
    * EIP - instruction ptr
    * Global descriptor table
* "OS Registers" - the registers that the OS has access to
    * Data segment
    * Stack segment
    * ICU - Interrupt Control Unit
    * IDT - Interrupt Descriptor Table (ptr to region of memory that contains)

**Memory**
* Stack
* Code
* = boundary =
* Kernel code
* Some tables that determine how processes see the hardware

What we have in the CPU constrains what we will be able to do
* eg. without a global descriptor table, we would have to map process tables differently

## CPU Modes

### Supervisor
* All processor functions can be accessed
    * Memory, devices, control tables (interrupt, page), registers
* Similar to root in UNIX

### User
* Restricted set to operations
    * some instructions (eg. one to flip modes) not allowed
    * no direct access to devices
    * cannot modify control tables used by the CPU
* Only access to general purpose registers
* Requests services from the kernels
* Similar to UNIX users

Part of setting up the kernel space region in memory is so that a user program generates a fault if it tries to access those regions (eg. segfault)

### CPU Modes allow us to...
* Conceptually treat the transition form user to supervisor as a red line and have two bodies of code
    * trusted kernel code
    * untrusted user mode
* Crossing the line must be highly controlled in both directions
* Going from user->kernel is done by syscalls (traps) and interrupts
* Going from kernel->user requires a privileged instruction

#### Syscall
1. User: User process executes
2. User: Calls syscall
3. TRAP! `mode bit = 0`
4. Kernel: execute syscall
5. Return! `mode bit = 1`
6. Return from system call
    